In [ ]:
import os
import re
filelist = os.listdir('logs')
labels = {
    'blackscholes':0,
    'bodytrack':1,
    'canneal':2,
    'freqmine':3,
    'vips':4,
    'streamcluster':5,
    'splash2x.radiosity':6,
    'splash2x.volrend':7,
    'splash2x.water_nsquared':8,
    'splash2x.water_spatial':9
}

In [ ]:
namereader = re.compile(r"(.+)_(\d+)_log\.txt")
m = namereader.match(filelist[5])
m.group(1)

In [ ]:
len(filelist)

In [ ]:
y_data = []
x_data = []
for filename in filelist:
    m = namereader.match(filename)
    if not m:
        continue
    label = labels[m.group(1)]
    trace = []
    with open('logs/'+filename,'r') as f:
        cnt = 0
        for line in f:
            if cnt == 0:
                cnt = 1
                continue
            tokens = line.split(" ")
            trace.append(float(tokens[1]))
            cnt += 1
    tracelen = len(trace)
    #offset = random.randint(0,tracelen-500)
    offset=0
    y_data.append(label)
    x_data.append(trace[offset:offset+500])

In [ ]:
split=800
from sklearn import svm
clf = svm.SVC(gamma='auto',kernel='linear')
clf.fit(x_data[split:], y_data[split:])
pred_y = clf.predict(x_data[:split])


In [ ]:
correct = [yp == y for yp,y in zip(pred_y, y_data[:split])]
sum(correct)/len(correct)

In [ ]:
pred_y[:20]

In [ ]:
y_data[:20]